# This notebook is a library
### It contains the followin functions:
1. **generate_data**: generates data for a learning process by means of simulation.

# Imports

In [1]:
import numpy as np

import tensorflow as tf

import keras
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Activation, MaxPooling1D, Dropout, BatchNormalization, Conv1D, Flatten
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2, l1_l2
from keras.metrics import mean_squared_error

from sklearn.model_selection import train_test_split


# Generate Data

In [3]:
def generate_data(N, M, nextConfig, sample):
    """
    This function generates data for a learning process by means of simulation.
    
    The data consists of ITEMS. Each ITEM is a pair: histogram and a configuration 
    (for now, the configuration is scalar / a floating point value).
    
    A histogram consists of sample points. 
    Each sample points is drawn from a distribution, with some specific configuration.
    
    Parameter setting for distribution = Configuration, eg.., alpha, gamma, alpha+LOC
    
    Returns:
        - a matrix of histograms and 
          an array of corresponding configs (one config for each histogram row).

    Arguments:
        - N: number of ITEMS to generate (rows)
        - M: number of sample points that make a histogram (columns)
        - nextConfig: A generator that gives a new alpha/gamma/delta, in each call.
            Example: nextConfig()
                lambda = random(1,10)
                return lambda
        - sample(config, size): returns sample data points from a distribution (size = num samples).
            Example sample(alpha, size=256) 
                ys = yulesimon(alpha, size)
                return ys

    """
    
    items_matrix = np.zeros((N, M), dtype=int)
    config_array = np.zeros((N,), dtype=float)
    
    # generate items_matrix:
    #   repeat N times:
    #     draw M sample points from distribution
    for i in range(N):
        
        # start item
        config = nextConfig()

        # sample data points with current config and add to sample_matrix
        items_matrix[i, :] = sample(config, size=M)
        
        # append corresponding config
        config_array[i] = config
    
    # create histograms from rows of items_matrix
    nbins = np.max(items_matrix)
#     histogram_matrix = np.apply_along_axis(
#         lambda a: np.histogram(a, bins=nbins)[0], 1, items_matrix)
#     histogram_matrix = np.apply_along_axis(
#         lambda a: np.histogram(a, bins=nbins, range=(1,np.max(a)))[0], 1, items_matrix)
    histogram_matrix = np.apply_along_axis(
        lambda a: np.histogram(a, bins=nbins, range=(1,nbins))[0], 1, items_matrix)

    return histogram_matrix, config_array
        

# Create DNN Model

In [ ]:
def create_dnn_model(n_features, 
                 layers, 
                 activation='relu', 
                 init='he_uniform', 
                 batch_normalization=False, 
                 dropout=0, 
                 optimizer='adam', 
                 k_reg=False, 
                 k_reg_lr=0.001, 
                 a_reg=False, 
                 a_reg_lr=0.001):

    model = Sequential()
    
    # ============
    # input-layer
    # ============
    model.add(Dense(units=layers[0]
                      , input_dim=n_features
                      , kernel_initializer=init
                      , kernel_regularizer=l2(k_reg_lr) if k_reg else None
                      , activity_regularizer=l2(a_reg_lr) if a_reg else None
                      , use_bias= (not batch_normalization)
                    ))
    
    
    if batch_normalization:
        model.add(BatchNormalization())
    
    model.add(Activation(activation))

    if dropout > 0:
        model.add(Dropout(dropout))

    # ==============
    # hidden-layers
    # ==============
    for units in layers[1:]:
        model.add(Dense(units=units
                        , kernel_initializer=init
                        , kernel_regularizer=l2(k_reg_lr) if k_reg else None
                        , activity_regularizer=l2(a_reg_lr) if a_reg else None
                        , use_bias= (not batch_normalization)
                        ))

    if batch_normalization:
        model.add(BatchNormalization())

    model.add(Activation(activation))
    
    if dropout > 0:
        model.add(Dropout(dropout))

    # =============
    # output-layer
    # =============
    model.add(Dense(units=1
                    , kernel_initializer=init
                    , kernel_regularizer=l2(k_reg_lr) if k_reg else None
                    , activity_regularizer=l2(a_reg_lr) if a_reg else None
                    , use_bias= (not batch_normalization)
                    ))
    
    if batch_normalization:
        model.add(BatchNormalization())

    model.add(Activation('linear'))

    model.compile(loss='mse', metrics=['mse'], optimizer=optimizer)

    return model

# Train DNN

In [ ]:
def train_dnn_model(model, X_train, y_train, batch_size=32):

    # split train/val
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

    # early-stopping
    es_patience = 50
    es = EarlyStopping(monitor='val_loss', 
                        patience=es_patience, 
                        mode='min', 
                        restore_best_weights=True, 
                        verbose=0)
    
    # reduce learning-rate on plateau
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=10)
    
    # fit model
    history = model.fit(X_train, 
                        y_train, 
                        validation_data=(X_val, y_val), 
                        epochs=200, 
                        batch_size=batch_size, 
                        shuffle=False, 
                        callbacks=[es, reduce_lr], 
                        verbose=0)
    
    # save history with model
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)
    
    # load best weights from last checkpoint
    model = keras.models.load_model(model_path)
    return model, history.history